In [ ]:
!pip install "transformers==4.40.1" utilsforecast

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import *

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import torch
from transformers import AutoModelForCausalLM

In [ ]:
model = AutoModelForCausalLM.from_pretrained('thuml/sundial-base-128m', trust_remote_code=True)

In [ ]:
DATA_URL = "https://raw.githubusercontent.com/marcopeix/FoundationModelsForTimeSeriesForecasting/refs/heads/main/data/Walmart.csv"

df = pd.read_csv(DATA_URL)
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

df.head()

## Zero-shot forecasting

In [ ]:
HORIZON = 13

In [ ]:
unique_stores = sorted(df['Store'].unique())
inputs_list = []

for store_id in unique_stores:
    store_df = df[df['Store'] == store_id].sort_values(by='Date')
    inputs_list.append(store_df['Weekly_Sales'].values)

input_tensor = torch.tensor(inputs_list, dtype=torch.float32)
print(input_tensor.shape)

In [ ]:
output = model.generate(
    input_tensor,
    max_new_tokens=HORIZON,
    num_samples=100
)
print(output.shape)

In [ ]:
def create_forecast_df(
    quantile_forecast_tensor: torch.Tensor,
    original_df: pd.DataFrame,
    id_col: str,
    time_col: str,
    target_col: str,
    horizon: int,
    freq: str,
):
    # Calculate quantiles across the num_samples dimension (dim=1)
    median_forecast = torch.quantile(quantile_forecast_tensor, 0.5, dim=1).numpy() # median
    lo_80_forecast = torch.quantile(quantile_forecast_tensor, 0.1, dim=1).numpy()  # 10th percentile
    hi_80_forecast = torch.quantile(quantile_forecast_tensor, 0.9, dim=1).numpy()  # 90th percentile

    num_series, _, horizon = quantile_forecast_tensor.shape

    all_forecast_rows = []
    unique_ids = sorted(original_df[id_col].unique())


    for i, id in enumerate(unique_ids):
        # Get the last known date for this id from the original DataFrame
        id_hist_df = original_df[original_df[id_col] == id].sort_values(by=time_col)
        last_known_date = id_hist_df[time_col].iloc[-1]

        # Generate forecast dates
        forecast_dates = pd.date_range(start=last_known_date, periods=horizon + 1, freq=freq)[1:]

        for h in range(horizon):
            forecast_row = {
                id_col: id,
                time_col: forecast_dates[h],
                'sundial': median_forecast[i, h],
                'sundial-lo-80': lo_80_forecast[i, h],
                'sundial-hi-80': hi_80_forecast[i, h]
            }
            all_forecast_rows.append(forecast_row)

    return pd.DataFrame(all_forecast_rows)


In [ ]:
fcsts_df = create_forecast_df(
    output,
    df,
    id_col='Store',
    time_col='Date',
    target_col='Weekly_Sales',
    horizon=HORIZON,
    freq='W-FRI'
)
fcsts_df.head()

In [ ]:
plot_series(
    df=df,
    forecasts_df=fcsts_df,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

## Cross-validation

In [ ]:
def sundial_cv(
    df: pd.DataFrame,
    model,
    horizon: int,
    n_windows: int,
    id_col: str,
    time_col: str,
    target_col: str,
    freq: str,
):
    all_cv_forecasts = []

    max_date = df[time_col].max()

    for i in range(n_windows):
        # Calculate the cutoff date for the current window
        cutoff_date = max_date - pd.Timedelta((n_windows - i) * horizon, unit=freq[0])

        # Create a training DataFrame up to the cutoff_date
        df_train = df[df[time_col] <= cutoff_date]

        # Prepare inputs_list
        unique_ids = sorted(df_train[id_col].unique())
        inputs_list = []
        for id in unique_ids:
            sub_df = df_train[df_train[id_col] == id].sort_values(by=time_col)
            inputs_list.append(sub_df[target_col].values)

        # Generate forecasts
        output = model.generate(
            torch.tensor(inputs_list, dtype=torch.float32),
            max_new_tokens=horizon,
            num_samples=100
        )

        # Convert forecasts to DataFrame
        fcsts_df = create_forecast_df(
            quantile_forecast_tensor=output,
            original_df=df_train,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
            horizon=horizon,
            freq=freq,
        )

        # Add cutoff column
        fcsts_df['cutoff'] = cutoff_date

        all_cv_forecasts.append(fcsts_df)

    cv_df = pd.concat(all_cv_forecasts, ignore_index=True)
    cv_df = cv_df.merge(df[[id_col, time_col, target_col]], how="left", on=[id_col, time_col])
    return cv_df

In [ ]:
cv_df = sundial_cv(
    df=df,
    model=model,
    horizon=HORIZON,
    n_windows=3,
    id_col='Store',
    time_col='Date',
    target_col='Weekly_Sales',
    freq="W-FRI",
)
cv_df.head()

In [ ]:
plot_series(
    df=df,
    forecasts_df=cv_df.drop(columns=["cutoff", "Weekly_Sales"]),
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

In [ ]:
eval_df = evaluate(
    cv_df.drop(columns=["cutoff"]),
    metrics=[mae, smape],
    models=['sundial'],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
eval_df